# Hidden Markov Model para glosado Otomi

In [1]:
from nltk.tag import hmm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from pickle import load
from itertools import chain
import pandas as pd
import numpy as np

In [2]:
file1 = load(open('pickle_objects/preinput_data','rb'))
print(file1[0], len(file1))

[[["'", 'v', 'B-como'], ['á', 'v', 'I-como'], ['b', 'v', 'B-3.cpl'], ['i', 'v', 'I-3.cpl'], ['m', 'v', 'B-ctrf'], ['á', 'v', 'I-ctrf'], ['k', 'v', 'B-stem'], ['h', 'v', 'I-stem'], ['á', 'v', 'I-stem']]] 1769


In [3]:
data = []

for s in file1:
    sent_data = []
    for w in s:
        #print(w)
        word = [(x[0], x[2]) for x in w] + [(' ', ' ')]
        sent_data.append(word)
    
    one_sent = list(chain(*sent_data))
    one_sent.pop(-1)
    data.append(list(zip(*one_sent)))

In [4]:
#Data split 33%
train_pairs, eval_pairs = train_test_split(data, test_size=0.1)#33)
print(len(train_pairs), len(eval_pairs))

#Separar el train en dos conjuntos
train_set = [list(zip(s,t)) for s,t in train_pairs]

1592 177


### Entrenamiento del modelo

In [5]:
%%time
trainer = hmm.HiddenMarkovModelTrainer()
tagger = trainer.train_supervised(train_set)

CPU times: user 89.9 ms, sys: 1.26 ms, total: 91.1 ms
Wall time: 88.9 ms


### Evaluación

In [6]:
y_pred = []
y_true = []
for sent in eval_pairs:
    prediccion = tagger.tag(sent[0])
    y_pred.append(prediccion)
    y_true.append(sent[1])

In [7]:
Y_true = list(chain(*y_true))
Y_pred = list(chain(*y_pred))
chars,Y_pred = zip(*Y_pred) 
Y_pred = list(Y_pred)

In [8]:
labels = list(set(Y_true))
labels.remove(' ')

print('Accuracy:', accuracy_score(Y_true, Y_pred))

Accuracy: 0.8787820015515904


In [9]:
prec, rec, f1, supp = precision_recall_fscore_support(Y_true, Y_pred, labels=labels)

/home/mijangos/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
results = pd.DataFrame(data=np.array([prec, rec, f1, supp]).T, index=labels, columns=['Precision','Recall','F1', 'Support'])
results

,Precision,Recall,F1,Support
B-1.icp.irr,0.333333,1.000000,0.500000,1.0
B-1.cpl,0.964286,1.000000,0.981818,27.0
B-stem,0.920000,0.843236,0.879947,791.0
I-3.prf,0.750000,1.000000,0.857143,3.0
B-aum,1.000000,1.000000,1.000000,2.0
...,...,...,...,...
B-3.pss,0.000000,0.000000,0.000000,2.0
B-1.icp,1.000000,0.964286,0.981818,28.0
B-lim,0.843750,0.931034,0.885246,29.0
I-1.obj,0.375000,0.900000,0.529412,10.0


In [11]:
print(results.to_string())

                  Precision    Recall        F1  Support
B-1.icp.irr        0.333333  1.000000  0.500000      1.0
B-1.cpl            0.964286  1.000000  0.981818     27.0
B-stem             0.920000  0.843236  0.879947    791.0
I-3.prf            0.750000  1.000000  0.857143      3.0
B-aum              1.000000  1.000000  1.000000      2.0
I-1.enf            0.800000  1.000000  0.888889      4.0
B-loc              0.714286  1.000000  0.833333      5.0
I-2.pot            1.000000  1.000000  1.000000      1.0
I-por.que          0.000000  0.000000  0.000000      2.0
I-2.cnt            1.000000  1.000000  1.000000      2.0
I-3.icp            0.000000  0.000000  0.000000      2.0
I-3.cpl            1.000000  1.000000  1.000000     52.0
B-dim              0.916667  0.916667  0.916667     12.0
I-loc              0.714286  1.000000  0.833333     10.0
I-ctrf             0.666667  1.000000  0.800000     26.0
B-2.pot            1.000000  1.000000  1.000000      1.0
B-mod              0.500000  0.

In [12]:
aver_prec = (supp*prec).sum()/supp.sum()
aver_rec = (supp*rec).sum()/supp.sum()
aver_f1 = (supp*f1).sum()/supp.sum()

print('Average precision: {}\nAverage recall: {}\nAverage F1: {}'.format(aver_prec, aver_rec, aver_f1))

Average precision: 0.8972871551219455
Average recall: 0.8587251356238698
Average F1: 0.8704548657812844
